# 1.分析数据结构

## 1.1 读取数据，分析有哪些column

In [1]:
import pandas as pd
import os
import time
import numpy as np
import re


In [2]:
def preview_parquet_structure(folder):
    for file in os.listdir(folder):
        if file.endswith(".parquet"):
            path = os.path.join(folder, file)
            df = pd.read_parquet(path, engine='pyarrow', columns=None)
            print(f"文件: {file}")
            print(df.info())
            print(df.head(2))
            break  # 只查看若干个样本文件即可


In [3]:
read_path = '10G_data_new'
read_file = 'part-00000.parquet'

start_time = time.time()
preview_parquet_structure(read_path)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


文件: part-00002.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5625000 entries, 0 to 5624999
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id                 int64  
 1   last_login         object 
 2   user_name          object 
 3   fullname           object 
 4   email              object 
 5   age                int64  
 6   income             float64
 7   gender             object 
 8   country            object 
 9   address            object 
 10  purchase_history   object 
 11  is_active          bool   
 12  registration_date  object 
 13  phone_number       object 
 14  login_history      object 
dtypes: bool(1), float64(1), int64(2), object(11)
memory usage: 606.2+ MB
None
         id                 last_login user_name fullname  \
0  11250000  2023-04-07T12:47:18+00:00    LMDCDN       仇波   
1  11250001  2024-06-23T10:53:27+00:00     IUOLI      胥泽霖   

                  email  age     income gender country   

In [4]:
read_path = '30G_data_new'
read_file = 'part-00000.parquet'

start_time = time.time()
preview_parquet_structure(read_path)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


文件: part-00008.parquet
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8437500 entries, 0 to 8437499
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id                 int64  
 1   last_login         object 
 2   user_name          object 
 3   fullname           object 
 4   email              object 
 5   age                int64  
 6   income             float64
 7   gender             object 
 8   country            object 
 9   address            object 
 10  purchase_history   object 
 11  is_active          bool   
 12  registration_date  object 
 13  phone_number       object 
 14  login_history      object 
dtypes: bool(1), float64(1), int64(2), object(11)
memory usage: 909.3+ MB
None
         id                 last_login user_name fullname  \
0  67500000  2023-11-12T11:56:50+00:00  SHRMHCDP       安娜   
1  67500001  2024-06-19T05:44:35+00:00  DUKMMLSQ      云雨泽   

                  email  age     income gender country   

1. 对于10G和30G，数据结构是一致的，只是规模不同，数据里面内容或许不同
1. 数据种类：bool(1), float64(1), int64(2), object(11)
1. 其中比较复杂的数据，有purchase_history，login_history，这两种数据预计为重点分析的数据
1. 用时有点长了，并且打印不完全，所以筛选一部分数据，保存为parquet/csv，更方便分析处理
1. 这里选择前1000条，进行少量数据分析，不直接全量分析，减少探索时间

## 1.2 挑选部分数据进行初步类型分析

In [5]:
def parquet2csv(path, p_file, c_file):
    os.makedirs(path, exist_ok=True)
    df = pd.read_parquet(os.path.join(path, p_file), engine='pyarrow', columns=None)
    df_ = df.head(1000)
    # 保存到csv文件（以便打开阅读）
    df_.to_csv(os.path.join(path, c_file))
    # 保存到新的 Parquet 文件
    output_path = os.path.join(path, "parquet-00000-1000.parquet")
    df_.to_parquet(output_path, engine='pyarrow')
    

In [6]:
read_path = '10G_data_new'
read_file = 'part-00000.parquet'
csv_file = 'part-00000-1000.csv'

start_time = time.time()
parquet2csv(read_path, read_file, csv_file)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


总计用时：39.67s


下面先具体分析一下数据结构，确认数据类型

In [10]:
import json

def inspect_parquet_row(file_path, row_index=0):
    print(f"读取文件: {file_path}")

    # 只读取指定行
    df = pd.read_parquet(file_path)
    row = df.iloc[row_index]
    print(f"\n=== 分析行 {row_index} ===\n")

    for col in df.columns:
        value = row[col]
        print(f"Column: {col}")
        print(f"  Type: {type(value)}")
        print(f"  Value: {str(value)[:300]}")  # 避免打印太长
        print()

        # 如果是 purchase_history 或 login_history，尝试 json.loads
        if col in ['purchase_history', 'login_history']:
            try:
                parsed = json.loads(value)
                print(f"  → Parsed JSON keys: {list(parsed.keys())}")
                if isinstance(parsed, dict):
                    for k, v in parsed.items():
                        print(f"    {k}: {type(v)} -> {str(v)[:100]}")
                print()
            except Exception as e:
                print(f"JSON 解码失败：{e}\n")


In [12]:
inspect_parquet_row("10G_data_new/parquet-00000-1000.parquet", row_index=0)


读取文件: 10G_data_new/parquet-00000-1000.parquet

=== 分析行 0 ===

Column: id
  Type: <class 'numpy.int64'>
  Value: 0

Column: last_login
  Type: <class 'str'>
  Value: 2024-12-02T03:49:12+00:00

Column: user_name
  Type: <class 'str'>
  Value: RKWKCXRZFV

Column: fullname
  Type: <class 'str'>
  Value: 瞿紫玉

Column: email
  Type: <class 'str'>
  Value: kuegujsk@hotmail.com

Column: age
  Type: <class 'numpy.int64'>
  Value: 82

Column: income
  Type: <class 'numpy.float64'>
  Value: 366311.83

Column: gender
  Type: <class 'str'>
  Value: 女

Column: country
  Type: <class 'str'>
  Value: 美国

Column: address
  Type: <class 'str'>
  Value: Non-Chinese Address Placeholder

Column: purchase_history
  Type: <class 'str'>
  Value: {"avg_price":9496,"categories":"零食","items":[{"id":7265}],"payment_method":"现金","payment_status":"已支付","purchase_date":"2023-07-30"}

  → Parsed JSON keys: ['avg_price', 'categories', 'items', 'payment_method', 'payment_status', 'purchase_date']
    avg_price: <class '

字段如下：

| 字段名 | 含义 | 类型 | 特征说明 |
|--------|------|------|---------|
| id | 用户唯一标识 | int64 | 无需处理，可直接使用 |
| last_login | 最近登录时间 | object (ISO时间) | 时间格式需转换 |
| user_name | 用户名 | object | 唯一性可检查，可脱敏 |
| fullname | 用户全名 | object | 可拆分姓与名 |
| email | 电子邮件 | object | 可提取邮箱域名 |
| age | 年龄 | int64 | 可检查异常值与分箱 |
| income | 收入 | float64 | 可标准化或分箱处理 |
| gender | 性别 | object | 类别型，需标准化编码 |
| country | 国家 | object | 类别型，合并小众国家 |
| address | 地址 | object | 可选处理，如地理编码 |
| purchase_history | 购买记录（JSON） | object | 需解析为结构化字段 |
| is_active | 是否活跃 | bool | 可直接使用 |
| registration_date | 注册时间 | object (ISO时间) | 时间格式需转换 |
| phone_number | 电话号码 | object | 可格式标准化，提取国家码 |
| login_history | 登录记录（JSO） | object | 需展

开提取如活跃天数、设备等特征 |

这里复制了一份涉及到两个复杂嵌套结构的东西
	purchase_history
{"avg_price":9496,"categories":"零食","items":[{"id":7265}],"payment_method":"现金","payment_status":"已支付","purchase_date":"2023-07-30"}

不是，等会儿，谁家零食吃了1w块？
不过这不是很重要了，主要是结构，这个里面的嵌套结构有
- avg_price, int
- categories, str
- items, list-dict
- payment_method, str
- payment_status, str
- purchase_date, datetime(yyyy-mm-dd)

	login_history
{"avg_session_duration":105,"devices":["desktop","mobile"],"first_login":"2024-12-04","locations":["home","travel"],"login_count":73,"timestamps":["2024-12-04 21:29:00","2024-12-12 20:51:00","2024-12-20 19:00:00","2024-12-28 10:58:00","2025-01-05 06:58:00","2025-01-13 21:55:00","2025-01-21 18:03:00","2025-01-29 18:26:00","2025-02-06 19:31:00","2025-02-14 11:15:00","2025-02-22 06:41:00","2025-03-02 10:10:00","2025-03-10 20:17:00","2025-03-18 20:19:00"]}

同理，login数据似乎看起来更长一些
- avg_session_duration, int
- devices, list-str
- first_login, datetime(yyyy-mm-dd)
- locations, list-str
- login_count, int
- timestamps, list-datetime(yyyy-mm-dd hh-mm-ss)

## 1.3 简单预处理


已经分析出各种数据的结构，那么就可以进行初筛，包括
- 缺失值分析
- 重复值分析
- 乱码
- 极端值
    - 年龄（<0 / >100）

至于其他的数据处理，比如异常值（性别为“武装直升机”），建议进行统计处理，比如实现一个class，统计一些类别的出现频次，然后可视化/分析。这些会在第二部分代码实现，因为已经涉及到对数据的分析处理了

理论上讲这些分析应该单独实现，但是问题是数据量实在是太大了，逐一分析会导致重复读取，浪费大量时间，因此选择实现一个汇总的代码，读取一次分析上述所有特殊情况

当然，这里不会直接进行处理，是先扫一遍，看看有没有出现上面的问题。如果有的话再决定后面怎么处理，直接删除/插值等方案都是备选

In [3]:
def is_garbled(s):
    """检测字符串是否包含明显乱码（非 ASCII + 非常见中日韩字符）"""
    if not isinstance(s, str):
        return False
    return bool(re.search(r'[^\u4e00-\u9fa5\u3040-\u30ff\uac00-\ud7af\w\s\.\,\-\+\@\:\(\)/]', s))


In [4]:
def analyze_parquet_files(folder):
    for file in os.listdir(folder):
        if file.endswith(".parquet"):
            print(f"正在分析: {file}")
            path = os.path.join(folder, file)
            df = pd.read_parquet(path, engine='pyarrow', columns=None)

            print(f"  - 总行数: {len(df)}")
    
            # 1. 缺失值分析
            sstart_time = time.time()
            print("1.缺失值分析")
            missing_counts = df.isnull().sum()
            missing_counts = missing_counts[missing_counts > 0]
            if not missing_counts.empty:
                print("每列缺失:")
                print(missing_counts.to_string())
            else:
                print("很好，无缺失值！")
            end_time = time.time()
            print(f"这一部分用时：{(end_time-sstart_time):.2f}s")
    
            # 2. 重复值分析
            start_time = time.time()
            print("2.重复值分析")
            duplicate_count = df.duplicated().sum()
            print(f"重复行数: {duplicate_count}")
            end_time = time.time()
            print(f"这一部分用时：{(end_time-start_time):.2f}s")
    
            # 3. 乱码检测（仅检测 object 类型）
            start_time = time.time()
            print("3.乱码检测")
            garbled_counts = {}
            for col in df.select_dtypes(include=['object']).columns:
                garbled_rows = df[col].dropna().apply(is_garbled)
                count = garbled_rows.sum()
                if count > 0:
                    garbled_counts[col] = count
            if garbled_counts:
                print("这里出现了乱码:")
                for col, count in garbled_counts.items():
                    print(f"    - {col}: {count} rows")
            else:
                print("很好！文件很标准，无乱码出现！")
            end_time = time.time()
            print(f"这一部分用时：{(end_time-start_time):.2f}s")
    
            # 4. 年龄极端值
            start_time = time.time()
            print("4.极端值-年龄")
            if 'age' in df.columns:
                extreme_ages = df[(df['age'] < 0) | (df['age'] > 100)]
                print(f"极端年龄 (<0 or >100): {len(extreme_ages)}")
            else:
                print("太好了，没有胚胎/修士出现在这份数据中！")
            end_time = time.time()
            print(f"这一部分用时：{(end_time-start_time):.2f}s")

            print(f"总计用时：{(end_time-sstart_time):.2f}s")
            print("-" * 60 + "\n")


In [5]:
start_time = time.time()
analyze_parquet_files("10G_data_new")
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


正在分析: part-00002.parquet
  - 总行数: 5625000
1.缺失值分析
很好，无缺失值！
这一部分用时：11.72s
2.重复值分析
重复行数: 0
这一部分用时：47.68s
3.乱码检测
这里出现了乱码:
    - purchase_history: 5625000 rows
    - login_history: 5625000 rows
这一部分用时：150.34s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：209.77s
------------------------------------------------------------

正在分析: part-00006.parquet
  - 总行数: 5625000
1.缺失值分析
很好，无缺失值！
这一部分用时：10.49s
2.重复值分析
重复行数: 0
这一部分用时：46.02s
3.乱码检测
这里出现了乱码:
    - purchase_history: 5625000 rows
    - login_history: 5625000 rows
这一部分用时：146.50s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：203.04s
------------------------------------------------------------

正在分析: part-00004.parquet
  - 总行数: 5625000
1.缺失值分析
很好，无缺失值！
这一部分用时：10.14s
2.重复值分析
重复行数: 0
这一部分用时：41.28s
3.乱码检测
这里出现了乱码:
    - purchase_history: 5625000 rows
    - login_history: 5625000 rows
这一部分用时：152.06s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：203.51s
------------------------------------------------------------

正在分析: part-00000.parquet
  - 总行

10G的数据用了接近33min，分析过程中主要耗时在乱码检测。检测到的乱码出现在最后一行，这个事情是有点怪的，因为我看到拿到的前1000行的csv是没有问题的

经过分析查证，应该和数据结构有关，最后一行会有文档结束标志，这个东西或许被识别为乱码了，而且恰好出现在两个嵌套结构中

所以以上涉及到的检测都没有问题，这份数据通过了预检测，目前看来不需要进行缺失值处理、异常值处理

In [ ]:
start_time = time.time()
analyze_parquet_files("30G_data_new")
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


正在分析: part-00008.parquet


30G的东西实在是太多了，2h分析notebook会kernel stop，所以我这里一个个执行

In [6]:
def analyze_parquet_files_list(file_list):
    for file_path in file_list:
        print(f"正在分析: {file_path}")
        df = pd.read_parquet(file_path, engine='pyarrow', columns=None)

        print(f"  - 总行数: {len(df)}")

        # 1. 缺失值分析
        sstart_time = time.time()
        print("1.缺失值分析")
        missing_counts = df.isnull().sum()
        missing_counts = missing_counts[missing_counts > 0]
        if not missing_counts.empty:
            print("每列缺失:")
            print(missing_counts.to_string())
        else:
            print("很好，无缺失值！")
        end_time = time.time()
        print(f"这一部分用时：{(end_time-sstart_time):.2f}s")

        # 2. 重复值分析
        start_time = time.time()
        print("2.重复值分析")
        duplicate_count = df.duplicated().sum()
        print(f"重复行数: {duplicate_count}")
        end_time = time.time()
        print(f"这一部分用时：{(end_time-start_time):.2f}s")

        # 3. 乱码检测（仅检测 object 类型）
        start_time = time.time()
        print("3.乱码检测")
        garbled_counts = {}
        for col in df.select_dtypes(include=['object']).columns:
            garbled_rows = df[col].dropna().apply(is_garbled)
            count = garbled_rows.sum()
            if count > 0:
                garbled_counts[col] = count
        if garbled_counts:
            print("这里出现了乱码:")
            for col, count in garbled_counts.items():
                print(f"    - {col}: {count} rows")
        else:
            print("很好！文件很标准，无乱码出现！")
        end_time = time.time()
        print(f"这一部分用时：{(end_time-start_time):.2f}s")

        # 4. 年龄极端值
        start_time = time.time()
        print("4.极端值-年龄")
        if 'age' in df.columns:
            extreme_ages = df[(df['age'] < 0) | (df['age'] > 100)]
            print(f"极端年龄 (<0 or >100): {len(extreme_ages)}")
        else:
            print("太好了，没有胚胎/修士出现在这份数据中！")
        end_time = time.time()
        print(f"这一部分用时：{(end_time-start_time):.2f}s")

        print(f"总计用时：{(end_time-sstart_time):.2f}s")
        print("-" * 60 + "\n")


In [7]:
start_time = time.time()
files = ["30G_data_new/part-00000.parquet", "30G_data_new/part-00001.parquet", "30G_data_new/part-00002.parquet", "30G_data_new/part-00003.parquet", ]
analyze_parquet_files_list(files)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


正在分析: 30G_data_new/part-00000.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：13.75s
2.重复值分析
重复行数: 0
这一部分用时：71.85s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：208.55s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.02s
总计用时：294.17s
------------------------------------------------------------

正在分析: 30G_data_new/part-00001.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：13.34s
2.重复值分析
重复行数: 0
这一部分用时：66.52s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：221.09s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.02s
总计用时：300.97s
------------------------------------------------------------

正在分析: 30G_data_new/part-00002.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：16.50s
2.重复值分析
重复行数: 0
这一部分用时：66.07s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：205.63s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：288.22s
------------------------------------------------------

In [8]:
start_time = time.time()
files = ["30G_data_new/part-00004.parquet", "30G_data_new/part-00005.parquet", "30G_data_new/part-00006.parquet", "30G_data_new/part-00007.parquet", ]
analyze_parquet_files_list(files)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


正在分析: 30G_data_new/part-00004.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：12.46s
2.重复值分析
重复行数: 0
这一部分用时：74.20s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：196.28s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.02s
总计用时：282.97s
------------------------------------------------------------

正在分析: 30G_data_new/part-00005.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：16.99s
2.重复值分析
重复行数: 0
这一部分用时：70.73s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：205.15s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.04s
总计用时：292.91s
------------------------------------------------------------

正在分析: 30G_data_new/part-00006.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：15.53s
2.重复值分析
重复行数: 0
这一部分用时：72.64s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：205.95s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：294.15s
------------------------------------------------------

In [9]:
start_time = time.time()
files = ["30G_data_new/part-00008.parquet", "30G_data_new/part-00009.parquet", "30G_data_new/part-00010.parquet", "30G_data_new/part-00011.parquet", ]
analyze_parquet_files_list(files)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


正在分析: 30G_data_new/part-00008.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：11.63s
2.重复值分析
重复行数: 0
这一部分用时：74.05s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：193.52s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.09s
总计用时：279.29s
------------------------------------------------------------

正在分析: 30G_data_new/part-00009.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：14.59s
2.重复值分析
重复行数: 0
这一部分用时：68.48s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：208.54s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.02s
总计用时：291.63s
------------------------------------------------------------

正在分析: 30G_data_new/part-00010.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：11.33s
2.重复值分析
重复行数: 0
这一部分用时：65.49s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：189.88s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：266.73s
------------------------------------------------------

In [10]:
start_time = time.time()
files = ["30G_data_new/part-00012.parquet", "30G_data_new/part-00013.parquet", "30G_data_new/part-00014.parquet", "30G_data_new/part-00015.parquet", ]
analyze_parquet_files_list(files)
end_time = time.time()
print(f"总计用时：{(end_time-start_time):.2f}s")


正在分析: 30G_data_new/part-00012.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：11.93s
2.重复值分析
重复行数: 0
这一部分用时：67.22s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：193.06s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.02s
总计用时：272.23s
------------------------------------------------------------

正在分析: 30G_data_new/part-00013.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：12.41s
2.重复值分析
重复行数: 0
这一部分用时：66.41s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：196.00s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.02s
总计用时：274.84s
------------------------------------------------------------

正在分析: 30G_data_new/part-00014.parquet
  - 总行数: 8437500
1.缺失值分析
很好，无缺失值！
这一部分用时：10.64s
2.重复值分析
重复行数: 0
这一部分用时：66.63s
3.乱码检测
这里出现了乱码:
    - purchase_history: 8437500 rows
    - login_history: 8437500 rows
这一部分用时：197.61s
4.极端值-年龄
极端年龄 (<0 or >100): 0
这一部分用时：0.03s
总计用时：274.91s
------------------------------------------------------

30G的数据时间更夸张，每一个文件5min，24个文件大概用了2h上下，分析过程中主要耗时在乱码检测。检测到的乱码出现在最后一行，这个事情是有点怪的，因为我看到拿到的前1000行的csv是没有问题的

经过分析查证，应该和数据结构有关，最后一行会有文档结束标志，这个东西或许被识别为乱码了，而且恰好出现在两个嵌套结构中

所以以上涉及到的检测都没有问题，这份数据通过了预检测，目前看来不需要进行缺失值处理、异常值处理